# Map Page

> The page of the app that generates the map view of the data.  

In [ ]:
#| default_exp pages/map_population

In [ ]:
#| export
# TODO minimise these imports a bit where possible
from nbdev.showdoc import *
# TODO work out how to get around below hack
try:
    from SolomonIslandsDataMap.dash_components import gen_bar_plot, gen_dd
    from SolomonIslandsDataMap.app_data import mytitle, map_graph, selectedBarGraph, stored_data, dropdown_location  \
        , control_type, dd_var, dd_measure, dd_geo_pop, sidebar_population
    from SolomonIslandsDataMap.load_data import SolomonGeo
except: 
    from dash_components import gen_bar_plot, gen_dd
    from app_data import mytitle, map_graph, selectedBarGraph, stored_data, dropdown_location \
        , control_type, dd_var, dd_measure, dd_geo_pop, sidebar_population
    from load_data import SolomonGeo
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio # Unless this is used graphs will not be dynamic?
import numpy as np
from fastcore.test import *
from dash import Dash, callback, dcc, Output, Input, State, html, Patch, ctx, register_page  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
from dash_bootstrap_templates import load_figure_template
import dash_mantine_components as dmc
import os
import json

In [ ]:
#| export
# Try catch is added here so that the notebook can be tested without the app being run
try:
    register_page(__name__, 
                        path='/',
                        title='Population Projections',
                        name='Population Projections')
except:
    pass

In [ ]:
#| export
# Creat some hacked init things
# TODO again as below, must be a better way
init_load = dcc.Store(id="initial-load", data={})
init_init = dcc.Store(id="initial-pop", data={})

Setup some of  the key data contiaining objects

## Customize the Layout

In [ ]:
#| export
def layout():
    return  dbc.Row(
        [dbc.Col(sidebar_population, width = 2),
        dbc.Col([
            mytitle,
            map_graph,
            #selectedBarGraph,
            stored_data, 
            init_load, 
            init_init,], width = 10),
            # fires 1ms after page load
            dcc.Interval(id="interval-timer", interval=1, max_intervals=1),
        ], justify = 'center')

### Some things to note about this page
- On Load, we change geo to province if it isn't alread there,
- We can just use the normal geo drowpdown because it is set to and will remain as Province, the other dropdown is basically just for display, 
- As a result, we can recycle a lot of the other dropdowns,
- Probably the only other one we will overwrite is the map one, to instead display population data. 
- It's really just updated to display new data not change to much of the other variables and callbacks

# Build callbacks to create interaction

#| hide
show_doc(update_measure)

## Update the variable dropdowns based on the selection of the overall variable

In [ ]:
#| export
@callback(
    Output(dd_measure, 'children', allow_duplicate=True),
    Input('varDropdown', 'value'),
    State('geo_df', 'data'),
    State('measureDropdown', 'value'),
    allow_duplicate=True,
    prevent_initial_call=True
)
def update_measure(new_var:str, # Selected variable
                   dict_sol:dict, # The dataset in dictionary form
                   measure:str, # Currently selected measure
              )->dcc.Dropdown: # Returns a dropdown of measures for selected variable
    '''
    Updates the dropdown_location dropdown based on the currently selected data aggregation.
    '''
    print("func um")
    sol_geo = SolomonGeo.gen_stored(dict_sol) # reload the data

    # Sometimes this callback is triggered when the measure doesn't need to be reset.
    # Check whether measure is in variable, if not reset to 0
    if measure not in sol_geo.census_vars[new_var]:
        measure = sol_geo.census_vars[new_var][0]

    # When a variable is selected, the measure will be set as the first one
    return gen_dd(sol_geo.census_vars[new_var], 'measureDropdown', 
                  val = measure)

In [ ]:
#| hide
show_doc(update_measure)

---

### update_measure

>      update_measure (new_var:str, dict_sol:dict, measure:str)

Updates the dropdown_location dropdown based on the currently selected data aggregation.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| new_var | str | Selected variable |
| dict_sol | dict | The dataset in dictionary form |
| measure | str | Currently selected measure |
| **Returns** | **Dropdown** | **Returns a dropdown of measures for selected variable** |

## Update the measure selection from clicks on the bar graph

In [ ]:
#| export
@callback(
    Output('measureDropdown', 'value', allow_duplicate=True),
    Output(selectedBarGraph, "clickData"),
    Input(selectedBarGraph, 'clickData'),
    State('varDropdown', 'value'),
    State('geo_df', 'data'),
    prevent_initial_call=True,
    allow_duplicate=True,
)
def bar_click(clickData:dict, # The currently clicked location on bar graph
                variable:str, # The currently selected variable
                dict_sol:dict, # The dataset in dictionary form
                )->[str]: # Returns the new value for the dropdown
    """This function updates the dropdown menu based on the bar graph click data"""
    print("func bc")
    sol_geo = SolomonGeo.gen_stored(dict_sol) # reload the data
    if clickData is None:
        print("Click data was none")
        return sol_geo.census_vars[variable][0], None
    else:
        # The measure are list of dictionaries
        selection = list(map(lambda x: x['x'], clickData['points']))[0]
    
        # returned objects are assigned to the component property of the Output
        # After updating fileter, we always reset map selection 
        return selection, None
        


In [ ]:
#| hide
show_doc(bar_click)

---

### bar_click

>      bar_click (clickData:dict, variable:str, dict_sol:dict)

This function updates the dropdown menu based on the bar graph click data

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| clickData | dict | The currently clicked location on bar graph |
| variable | str | The currently selected variable |
| dict_sol | dict | The dataset in dictionary form |
| **Returns** | **[<class 'str'>]** | **Returns the new value for the dropdown** |

### Update map based on inputs

In [ ]:
#| export
@callback(
    Output(map_graph, 'figure', allow_duplicate=True),
    Output(mytitle, 'children'),
    Input(dropdown_geo, 'value'),
    Input("segmented_type", 'value'),
    Input('measureDropdown', 'value'),
    Input('varDropdown', 'value'),
    Input('initial-load', 'data'),
    State('geo_df', 'data'),
    allow_duplicate=True,
    prevent_initial_call=True)
def update_map(geo_input:str, # User input from the geography dropdown
                data_type:str, # User input of type of data
                measure:str, # A string contiaining the census variable and measure split by ':'
                variable:str, # The state of the variable dropdown
                init_load:{}, # An empty dictionary always
                dict_sol:dict, # The dataset in dictionary form
              )->(type(go.Figure()), str): # Returns a graph object figure after being updated and the dynamic title
    '''
    Updates the focus census variable or geography dispalayed on the map
    '''
    # TODO the None workaround might be taxing on the load times, is there a better way
    # or maybe I can check it it needs updating?
    patched_figure = Patch()
    button_clicked = ctx.triggered_id

    sol_geo = SolomonGeo.gen_stored(dict_sol) # reload the data
    print("first run, updating map")
    print(button_clicked)
    print(geo_input)
    print(data_type)
    print(measure)

    # A None value is passed when the page is first loaded, hence
    # the the values are reset.
    if button_clicked in [dropdown_geo.id, dropdown_location.id, 'initial-load']:
        # Update disaplayed geography 
        for geo in sol_geo.geo_levels:
            tn = np.where(sol_geo.geo_levels == geo)[0][0] # Tracks the trace number
            patched_figure['data'][tn]['visible'] = geo_input == geo
            print(geo)
            print(geo_input == geo)
        
    if button_clicked in [control_type.id, 'initial-load']:
        # Update the type of data displayed on map and the hover template
        for geo in sol_geo.geo_levels:
            tn = np.where(sol_geo.geo_levels == geo)[0][0] # Tracks the trace number
            ar = sol_geo.get_census(geo_filter = geo, type_filter=data_type, var = variable, measure = measure).values
            ar = ar.reshape((ar.shape[0],))
            if data_type == 'Total':
                ht = '%{customdata} <extra>%{z}</extra>'
            elif data_type == 'Proportion':
                ht = '%{customdata} <extra>%{z:.1%}</extra>'
            else:
                ValueError("Data type of map not recognised and note accounted for")
            patched_figure['data'][tn]['z'] = ar
            patched_figure['data'][tn]['hovertemplate'] = ht

            
        

    if button_clicked in ['measureDropdown', 'initial-load']:
        # Update the z values in map to the data for the requested census variable
        for geo in sol_geo.geo_levels:
        # Ar updates the z value ie. data disaplyed each time
        # TODO this is fairly inefficient, as we are processing each time
        # Maybe faster framework like polars could help? or caching but would require a lot of caching
            tn = np.where(sol_geo.geo_levels == geo)[0][0] # Tracks the trace number
            ar = sol_geo.get_census(geo_filter = geo, type_filter=data_type, var = variable, measure=measure).values
            ar = ar.reshape((ar.shape[0],))
            patched_figure['data'][tn]['z'] = ar
        
    # returned objects are assigned to the component property of the Output
    # After updating fileter, we always reset map selection 

    return patched_figure, '## Solomon Islands Data map - ' + geo_input

In [ ]:
#| hide
show_doc(update_map)

---

### update_map

>      update_map (geo_input:str, data_type:str, measure:str, variable:str,
>                  init_load:{}, dict_sol:dict)

Updates the focus census variable or geography dispalayed on the map

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| geo_input | str | User input from the geography dropdown |
| data_type | str | User input of type of data |
| measure | str | A string contiaining the census variable and measure split by ':' |
| variable | str | The state of the variable dropdown |
| init_load | {} | An empty dictionary always |
| dict_sol | dict | The dataset in dictionary form |
| **Returns** | **(<class 'plotly.graph_objs._figure.Figure'>, <class 'str'>)** | **Returns a graph object figure after being updated and the dynamic title** |

### Update the selected part of the map based on the dropdown

### Update Selected Bar Graph based on inputs

In [ ]:
#| export
# Callback allows components to interact
@callback(
    Output(selectedBarGraph, 'figure'),
    Input("segmented_type", 'value'),
    Input('measureDropdown', 'value'),
    Input('locDropdown', 'value'),
    Input('initial-load', 'data'),
    State(dropdown_geo, 'value'),
    State('varDropdown', 'value'),
    State('geo_df', 'data'),
    allow_duplicate=True,
    prevent_initial_call=True
)
def update_bargraph(data_type:str, # User input of type of data
                     measure:str, # A string contiaining the census variable and measure split by ':'
                     loc_selection:[str], # The selected locations, may be none
                     init_load:{}, # An empty dictionary always, triggers initial load
                     geo_input:str, # User input from the geography dropdown
                     variable:str, # The state of the variable dropdown
                     dict_sol:dict, # The dataset in dictionary form
              )->(type(go.Figure())): # Returns a graph object figure after being updated and the dynamic title
    '''
    Updates the focus census variable or geography dispalayed on the map
    '''
    sol_geo = SolomonGeo.gen_stored(dict_sol) # reload the data

    # Create newly selected barplot
    print("input")
    print(loc_selection)
    locs = []
    # Multi dropdown can return None or a list of None.
    if len(loc_selection) > 0: 
        locs = loc_selection
    print("Going in to function")
    print(locs)
    bg = gen_bar_plot(sol_geo, geo_input, variable, measure, locs, data_type)

    # returned objects are assigned to the component property of the Output
    # After updating fileter, we always reset map selection 

    return  bg

On click, update the contents of the kpi's

In [ ]:
#| hide
show_doc(update_geography)

---

### update_geography

>      update_geography (geo_input:str, dict_sol:dict, locations:[<class'str'>])

Updates the dropdown_location dropdown based on the currently selected data aggregation.
Check to see if current locations are in geography, if they are not then reset them.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| geo_input | str | User input from the geography dropdown |
| dict_sol | dict | The dataset in dictionary form |
| locations | [<class 'str'>] | Currently selected locations |
| **Returns** | **[<class 'str'>]** | **Returns a new list of locations to display** |

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()